<a href="https://colab.research.google.com/github/Jack3690/pydol/blob/main/notebooks/pydol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jwst -q
!pip install crds -q
!pip install astroquery -q

In [ ]:
!pip install wfc3tools -q
!pip install stwcs -q
!pip install drizzlepac -q

In [ ]:
!pip install photutils==1.11.0 -q

In [ ]:
!git clone https://github.com/Jack3690/pydol

# **Pipeline**

In [1]:
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Image3Pipeline
import jwst.associations

from glob import glob

import os
from crds import client
import jwst
import multiprocessing as mp
from pathlib import Path

In [2]:
from wfc3tools import calwf3
from stwcs import updatewcs
from drizzlepac import astrodrizzle



The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol


In [4]:
class jpipe():
    def __init__(self, input_files=[], out_dir='.',
                 crds_context="jwst_1258.pmap", crds_dir='.'):
        """
            Parameters
            ----------
            input_files: list,
                         Input list of level 0 '_uncal.fits' files.
                         Recommended: /data/stage0/
            out_dir: str,
                     Output directory.
                     Recommended: The directory that contains /data/stage0/
                     Pipeline will create /data/stage1/ and /data/stage2/

            crds_context: str,
                          Reference context for JWST pipeline from CRDS.

              Returns
              -------
                  None

        """
        client.set_crds_server("https://jwst-crds.stsci.edu")
        if os.access(crds_dir,os.W_OK):
            crds_dir += '/CRDS/'
            os.makedirs(crds_dir, exist_ok=True)
        else:
            raise Exception(f"{crds_dir} is not WRITABLE")

        os.environ['CRDS_PATH'] = str(crds_dir)
        os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
        if len(input_files)<1:
            raise Exception("Input files list CANNOT be empty!")
        self.input_files = input_files
        self.out_dir = out_dir
        if os.access(out_dir,os.W_OK):
            os.makedirs(out_dir + '/stage1/', exist_ok=True)
            os.makedirs(out_dir + '/stage2/', exist_ok=True)
            os.makedirs(out_dir + '/stage3/', exist_ok=True)
        else:
            raise Exception(f"{out_dir} is not WRITABLE")

        os.environ["CRDS_CONTEXT"] = crds_context

    def stage1_pipeline(self, filename):
        """
            Parameters
            ----------
            filename: str,
                      path to the level 0 "_uncal.fits" file
            Returns
            -------
                None
        """
        # Instantiate the pipeline
        img1 = Detector1Pipeline()
        # Snowball Removal (M82 Group)
        img1.jump.expand_large_events = True
        # Specify where the output should go
        img1.output_dir = self.out_dir + '/stage1/'
        # Save the final resulting _rate.fits files
        img1.save_results = True
        #No of cores
        img1.jump.maximum_cores = f'{mp.cpu_count()-1}'
        # Run the pipeline on an input list of files
        img1(filename)

    def stage2_pipeline(self, filename):
        """
            Parameters
            ----------
            filename: str,
                      path to the level 1 "_rate.fits" file
            Returns
            -------
                None
        """
        # Instantiate the pipeline
        img2 = Image2Pipeline()
        # Specify where the output should go
        img2.output_dir = self.out_dir + '/stage2/'
        # Save the final resulting _rate.fits files
        img2.save_results = True
        # Run the pipeline on an input list of files
        img2(filename)

        # TBD: Add 1/f noise subtraction
        #
        #
        #
        #

    def stage3_pipeline(self, filenames):
        """
            Parameters
            ----------
            filename: str,
                      list of paths to the level 2 "_cal.fits" files

                      if a single file is provided only
                      resample and source_catalog steps will be applied.
            Returns
            -------
                None
        """
        # Instantiate the pipeline
        img3 = Image3Pipeline()
        # Specify where the output should go
        img3.output_dir = self.out_dir + '/stage3/'
        # Save the final resulting _rate.fits files
        img3.save_results = True
        # Run the pipeline on an input list of files
        img3(filenames)

    def __call__(self):
        """
            Runs the JWST Stage 1, Stage 2, and Stage 3 pipeline for generating
            '_crf.fits' files
        """
        # Stage1
        uncal_files = [i for i in self.input_files if 'uncal' in i ]
        uncal_names = [i.split('/')[-1] for i in uncal_files]

        rate_files = [self.out_dir + '/stage1/' + i.replace('uncal', 'rate') \
                      for i in uncal_names]

        [ self.stage1_pipeline(f) for n,f in enumerate(uncal_files)\
         if not os.path.exists(rate_files[n]) ]

        # Stage 2
        rate_files_ = [f for f in rate_files if not os.path.exists(f.replace('stage1', 'stage2').replace('rate', 'cal'))]

        if len(rate_files_)>0:
            with mp.Pool(mp.cpu_count()-1) as p:
                p.map(self.stage2_pipeline, rate_files_)

        # Stage 3
        cal_files = [f.replace('stage1', 'stage2').replace('rate', 'cal') for f in rate_files]
        cal_files_ = [f for f in cal_files if not os.path.exists(f.replace('stage2', 'stage3').replace('cal', 'crf'))]

        if len(cal_files_) > 0:
            self.stage3_pipeline(cal_files)

In [5]:
class hstpipe():
    def __init__(self, input_files=[], out_dir='.',
                 crds_context="jwst_1258.pmap", crds_dir='.'):
        """
            Parameters
            ----------
            input_files: list,
                         Input list of level 0 '_uncal.fits' files.
                         Recommended: /data/stage0/
            out_dir: str,
                     Output directory.
                     Recommended: The directory that contains /data/stage0/
                     Pipeline will create /data/stage1/ and /data/stage2/

            crds_context: str,
                          Reference context for JWST pipeline from CRDS.

              Returns
              -------
                  None

        """


        client.set_crds_server("https://hst-crds.stsci.edu")
        if os.access(crds_dir,os.W_OK):
            crds_dir += '/CRDS/'
            os.makedirs(crds_dir, exist_ok=True)
        else:
            raise Exception(f"{crds_dir} is not WRITABLE")

        os.environ['CRDS_SERVER_URL'] = 'https://hst-crds.stsci.edu'
        os.environ['CRDS_SERVER'] = 'https://hst-crds.stsci.edu'

        if len(input_files)<1:
            raise Exception("Input files list CANNOT be empty!")

        self.input_files = input_files

        for file in input_files:
          command_line_input = 'crds bestrefs --files {:} --sync-references=1 --update-bestrefs'.format(file)
          os.system(command_line_input)

        self.out_dir = out_dir
        if os.access(out_dir,os.W_OK):
            os.makedirs(out_dir + '/stage1/', exist_ok=True)
            os.makedirs(out_dir + '/stage2/', exist_ok=True)
            os.makedirs(out_dir + '/stage3/', exist_ok=True)
        else:
            raise Exception(f"{out_dir} is not WRITABLE")

    def stage2_pipeline(self, filenames):
        """
            Parameters
            ----------
            filename: str,
                      path to the level 0 "_uncal.fits" file
            Returns
            -------
                None
        """
        calwf3(filenames)

        flt_files = glob('./*_flt.fits')
        flt_files_n = []

        for f in flt_files:
          flt_n = self.out_dir + '/stage2/' + i.split('/')[-1]
          flt_files_n.append(flt_n)
          os.rename(f, flt_n)

        for flt in flt_files_n:
          updatewcs.updatewcs(input=flt)

    def stage3_pipeline(self, filenames):
        """
            Parameters
            ----------
            filename: str,
                      path to the level 1 "_rate.fits" file
            Returns
            -------
                None
        """
        astrodrizzle.AstroDrizzle(filenames)


    def __call__(self):
        """
            Runs the HST CALWF3 pipeline and astrodrizzle for generating
            '_crf.fits' files and drz image
        """
        # Stage2
        raw_files = [i for i in self.input_files if 'raw' in i]
        self.stage1_pipeline(raw_files)

        flt_names = [f.replace('raw', 'flt') for f in self.input_files]
        flt_files = [self.out_dir + '/stage1/' + i for i in flt_names]

        # Stage 3
        if len(flt_files) > 0:
            self.stage3_pipeline(flt_files)

# **Photometry**

In [6]:
import os
from glob import glob
from astropy.table import Table
from astropy.wcs import WCS
from astropy.io import fits
import numpy as np
import multiprocessing as mp
from pathlib import Path
import subprocess

param_dir_default = '/content/pydol/src/pydol/photometry/params'# str(Path(__file__).parent.joinpath('params'))
script_dir = '/content/pydol/src/pydol/photometry/scripts'#str(Path(__file__).parent.joinpath('scripts'))

In [7]:
def nircam_phot(cal_files, filter='f200w',output_dir='.', drz_path='.',
                cat_name='', param_file=None,sharp_cut=0.01,
                crowd_cut=0.5):
    """
        Parameters
        ---------
        cal_files: list,
                    list of paths to JWST NIRCAM level 2 _cal.fits files
        filter: str,
                name of the NIRCAM filter being processed
        output_dir: str,
                    path to output directory.
                    Recommended: /photometry/
        drz_path: str,
                  path to level 3 drizzled image (_i2d.fits) image.
                  It is recommended to be inside /photometry/
        cat_name: str,
                  Output photometry catalogs will have prefix filter + cat_name

        Return
        ------
        None
    """
    if len(cal_files)<1:
        raise Exception("cal_files cannot be EMPTY")

    subprocess.run([f"nircammask {drz_path}.fits"], shell=True)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    if param_file is None or not os.path.exists(param_file) :
      print("Using Default params")
      edit_params = True
      param_file = param_dir_default + '/nircam_dolphot.param'
    else:
      edit_params = False

    out_id = filter + cat_name

    # Generating directories
    exps = []
    for i,f in enumerate(cal_files):
        out_dir = f.split('/')[-1].split('.')[0]

        if not os.path.exists(f'{output_dir}/{out_dir}'):
            os.mkdir(f'{output_dir}/{out_dir}')
        if not os.path.exists(f"{output_dir}/{out_dir}/data.fits"):
            subprocess.run([f"cp {f} {output_dir}/{out_dir}/data.fits"],
                                shell=True)

        exps.append(f'{output_dir}/{out_dir}')

    # Applying NIRCAM Mask
    print("Running NIRCAMMASK and CALCSKY...")
    for f in exps:
        if not os.path.exists(f"{f}/data.sky.fits"):
            out = subprocess.run([f"nircammask {f}/data.fits"]
                                    ,shell=True)

            out = subprocess.run([f"calcsky {f}/data 10 25 2 2.25 2.00"]
                                , shell=True, capture_output=True)
    if edit_params:
      # Preparing Parameter file DOLPHOT NIRCAM
      with open(param_file) as f:
                  dat = f.readlines()

      dat[0] = f'Nimg = {len(exps)}                #number of images (int)\n'
      dat[4] = f'img0_file = {drz_path}\n'
      dat[5] = ''

      for i,f in enumerate(exps):
          dat[5] += f'img{i+1}_file = {f}/data           #image {i+1}\n'

      with open(f"{output_dir}/nircam_dolphot_{out_id}.param", 'w', encoding='utf-8') as f:
          f.writelines(dat)
      param_file = f"{output_dir}/nircam_dolphot_{out_id}.param"
    if not os.path.exists(f"{output_dir}/{out_id}_photometry.fits"):
        # Running DOLPHOT NIRCAM
        p = subprocess.Popen(["dolphot", f"{output_dir}/out", f"-p{param_file}"]
                            , stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                             text=True)
        while (line := p.stdout.readline()) != "":
          print(line)
    # Generating Astropy FITS Table

        out = subprocess.run([f"python {script_dir}/to_table.py --o {out_id}_photometry --n {len(exps)} --f {output_dir}/out"],
                       shell=True)

        phot_table = Table.read(f"{output_dir}/{out_id}_photometry.fits")

        # Assingning RA-Dec using reference image
        hdu = fits.open(f"{drz_path}.fits")[1]

        wcs = WCS(hdu.header)
        positions = np.transpose([phot_table['x'] - 0.5, phot_table['y']-0.5])

        coords = np.array(wcs.pixel_to_world_values(positions))

        phot_table['ra']  = coords[:,0]
        phot_table['dec'] = coords[:,1]

        # Filtering stellar photometry catalog using Warfield et.al (2023)
        phot_table1 = phot_table[ (phot_table['obj_sharpness']**2<= sharp_cut) &
                                    (phot_table['obj_crowd']<= crowd_cut) &
                                    (phot_table['type'] <= 2)]
        flag_keys = []
        for key in phot_table1.keys():
            if 'flag' in key:
                flag_keys.append(key)
        for i in flag_keys:
            phot_table1  = phot_table1[phot_table1[i]<=2]

        SNR_keys = []
        for key in phot_table1.keys():
            if 'SNR' in key:
                SNR_keys.append(key)
        for i in SNR_keys:
            phot_table1  = phot_table1[phot_table1[i]>=5]

        phot_table.write(f'{output_dir}/{out_id}_photometry.fits', overwrite=True)
        phot_table1.write(f'{output_dir}/{out_id}_photometry_filt.fits', overwrite=True)
    print('NIRCAM Stellar Photometry Completed!')

# **Testing**

In [ ]:
!wget http://americano.dolphinsim.com/dolphot/dolphot2.0.tar.gz
!wget http://americano.dolphinsim.com/dolphot/dolphot2.0.NIRCAM.tar.gz
!wget http://americano.dolphinsim.com/dolphot/nircam_F200W.tar.gz

--2024-08-25 17:22:22--  http://americano.dolphinsim.com/dolphot/dolphot2.0.tar.gz
Resolving americano.dolphinsim.com (americano.dolphinsim.com)... 64.139.89.252
Connecting to americano.dolphinsim.com (americano.dolphinsim.com)|64.139.89.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 270579 (264K) [application/x-gzip]
Saving to: ‘dolphot2.0.tar.gz’

dolphot2.0.tar.gz   100%[===================>] 264.24K   353KB/s    in 0.7s    

2024-08-25 17:22:23 (353 KB/s) - ‘dolphot2.0.tar.gz’ saved [270579/270579]

--2024-08-25 17:22:23--  http://americano.dolphinsim.com/dolphot/dolphot2.0.NIRCAM.tar.gz
Resolving americano.dolphinsim.com (americano.dolphinsim.com)... 64.139.89.252
Connecting to americano.dolphinsim.com (americano.dolphinsim.com)|64.139.89.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82691 (81K) [application/x-gzip]
Saving to: ‘dolphot2.0.NIRCAM.tar.gz’

dolphot2.0.NIRCAM.t 100%[===================>]  80.75K   195KB/s   

In [ ]:
!tar -xf dolphot2.0.tar.gz
!tar -xf dolphot2.0.NIRCAM.tar.gz
!tar -xf nircam_F200W.tar.gz

In [ ]:
with open('/content/dolphot2.0/Makefile') as f:
  dat = f.readlines()
dat[28]= dat[28][1:]

with open('/content/dolphot2.0/Makefile','w') as f:
  f.writelines(dat)

In [ ]:
cd /content/dolphot2.0/

In [ ]:
import os

In [ ]:
os.system('make all')

In [ ]:
cd /content/

In [ ]:
os.environ['PATH'] += ':/content/dolphot2.0/bin'
os.environ['PATH'] += ':/content/dolphot2.0/nircam'

In [ ]:
from glob import glob
import astropy.io.fits as fits

import os
import astroquery
from astroquery.mast import Observations
import pandas as pd

from astropy.table import Table, unique, vstack
from astropy.wcs import WCS
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
import json
from astropy.coordinates import angular_separation
import astropy.units as u
import multiprocessing as mp

In [ ]:
#Example data for F200W filter for NGC628
obs_table = Observations.query_criteria(proposal_id=["1783"],objectname="NGC628",filters= ['F115W', 'F150W', 'F200W'])[:3]

t = [Observations.get_product_list(obs) for obs in obs_table]
files = unique(vstack(t), keys='productFilename')

In [ ]:
mast_dir = 'mast:jwst/product' # Download from MAST
data_dir = '/content/data/stage0/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

manifest = Observations.download_products(
           files[:10],
           productSubGroupDescription='UNCAL',
           download_dir = data_dir,
           flat = True,
           )

In [ ]:
#Example data for NGC628

mast_dir = 'mast:jwst/product' # Download from MAST
data_dir = '/content/photometry/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

manifest = Observations.download_products(
           files,
           productSubGroupDescription='I2D',
           filters=['F200W'],
           calib_level=[3],
           download_dir = data_dir,
           flat = True,
           )

In [ ]:
input_files = glob(f'./data/stage0/*')
out_dir = './data'
jwst_data = jpipe(input_files, out_dir,crds_dir='.')
jwst_data()

In [ ]:
crf_files = glob(f'/content/data/stage3/*crf.fits')

In [ ]:
nircam_phot(crf_files, 'f212n','/content/photometry',
                          '/content/photometry/jw01783-o004_t008_nircam_clear-f200w_i2d')

In [ ]:
# to_table.py

with open('../photometry/out.columns') as f
  cols = f.readlines()
  for n, i in enumerate(cols):
    if 'Measured' in i:
      print(n)
      n_filt = (n - 11)//13
      break

filts = [cols[11+ i*13].split('NIRCAM_')[-1][:-1] for i in range(n_filt)]

col_source = ['ext','chip','x','y','chi_fit','obj_SNR','obj_sharpness','obj_roundness','dir_maj_axis','obj_crowd','type',]
col_filt = ['counts_tot','sky_tot','count_rate','count_rate_err','mag_vega','mag_ubvri','mag_err','chi','SNR','sharpness','roundness','crowd','flags']

out_cols = col_source
for i in filts:
    for j in col_filt:
        out_cols.append(j + '_' + i)

with open('../photometry/out') as f:
    dat = f.readlines()
data = np.array([i.split()[:len(out_cols)] for i in dat]).astype(float)